In [32]:
import pandas as pd
import numpy as np
import cv2
import os
from ultralytics import YOLO

In [33]:
model = YOLO('../models/yolo/yolov8m.pt')

In [35]:
import cv2
import numpy as np
from datetime import datetime
from torchvision import transforms
from PIL import Image

class PredPipe():
    def __init__(self, model, image_path, noise, blur):
        self.model = model
        self.image_path = image_path
        self.image = cv2.imread(image_path)
        self.name = image_path.split('/')[-1].split('.')[0]
        self.results = {}
        self.model_scores = []
        self.noise_and_blur = {
            'noise_pctg': noise,
            'blur_pctg': blur
        }
        self.timestamp = str(int(datetime.timestamp(datetime.now())))

    def add_noise_and_blur(self, save=False):
        image = cv2.imread(self.image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = image.astype(np.float32) / 255.0

        noise = np.random.uniform(low=0.0, high=1.0, size=image.shape)
        noise_mask = np.random.binomial(1, self.noise_and_blur['noise_pctg'], size=image.shape[:-1])
        noise_mask = np.expand_dims(noise_mask, axis=-1)
        noise_mask = np.repeat(noise_mask, 3, axis=-1)
        image = noise_mask * noise + (1 - noise_mask) * image
        image = (image * 255.0).astype(np.uint8)
        noisy_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if save:
            noisy_image_path = f"../data/processed/noisy/{self.name}_{str(self.noise_and_blur['noise_pctg'])}_noisy_{self.timestamp}.jpg"
            cv2.imwrite(noisy_image_path, noisy_image)

        return image, noisy_image_path

    def save_traffic_lights(self, image, result, image_path, timestamp):
        output_folder = "../data/processed/outputs/semafaros/standard/"
        traffic_lights = []  # Lista para salvar os semáforos detectados

        for box in result.boxes:
            class_id = result.names[box.cls[0].item()]
            if class_id == "traffic light":
                cords = box.xyxy[0].tolist()
                cords = [int(round(x)) for x in cords]

                # Ajuste as coordenadas para garantir que estejam dentro dos limites da imagem
                cords[0] = max(0, cords[0])
                cords[1] = max(0, cords[1])
                cords[2] = min(image.shape[1], cords[2])
                cords[3] = min(image.shape[0], cords[3])

                traffic_light_image = image[cords[1]:cords[3], cords[0]:cords[2]]

                if traffic_light_image.shape[0] > 0 and traffic_light_image.shape[1] > 0:
                    # Certifique-se de que a região recortada não é vazia
                    traffic_lights.append(traffic_light_image)

        if not traffic_lights:
            raise ValueError("Nenhum semáforo detectado na imagem.")

        # Nome da foto original
        image_name = os.path.splitext(os.path.basename(image_path))[0]
        
        for i, traffic_light_image in enumerate(traffic_lights):
            # Se houver mais de um semáforo na mesma foto, inclua um índice entre parênteses
            if i > 0:
                traffic_light_filename = f"traffic_light_{image_name}({i}).jpg"
            else:
                traffic_light_filename = f"traffic_light_{image_name}.jpg"
            
            traffic_light_path = os.path.join(output_folder, traffic_light_filename)
            
            # Verifique se o arquivo já existe antes de salvá-lo
            if not os.path.exists(traffic_light_path):
                cv2.imwrite(traffic_light_path, traffic_light_image)
                print(f"Semáforo {i + 1} da imagem {image_name} salvo em {traffic_light_path}")
            else:
                print(f"Semáforo {i + 1} da imagem {image_name} já existe em {traffic_light_path}. Não foi salvo novamente.")


    def predict_image(self, add_noise=False, save=True):
        if add_noise:
            image, noisy_image_path = self.add_noise_and_blur(save)
            results = self.model.predict(noisy_image_path)
            result = results[0]
            if save:
                predicted_image_path = f"../data/processed/outputs/{self.name}_{str(self.noise_and_blur['noise_pctg'])}_noisy_predicted_{self.timestamp}.jpg"
        else:
            results = self.model.predict(self.image_path)
            result = results[0]
            if save:
                predicted_image_path = f"../data/processed/outputs/{self.name}_predicted_{self.timestamp}.jpg"

        img = cv2.imread(self.image_path) if not add_noise else cv2.imread(noisy_image_path)

        for box in result.boxes:
            class_id = result.names[box.cls[0].item()]
            cords = box.xyxy[0].tolist()
            cords = [int(round(x)) for x in cords]
            conf = round(box.conf[0].item(), 2)

            # Desenhe um rótulo com confiança, mas sem bordas verdes
            label = f"{class_id} {conf}"
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            font_thickness = 1
            font_color = (0, 0, 255)
            background_color = (0, 0, 0)
            label_size, baseline = cv2.getTextSize(label, font, font_scale, font_thickness)
            label_origin = (cords[0], cords[1] - label_size[1])
            cv2.rectangle(img, (cords[0], cords[1]), (cords[0] + label_size[0], cords[1] - label_size[1]), background_color, -1)
            cv2.putText(img, label, label_origin, font, font_scale, font_color, font_thickness)

        if save:
            cv2.imwrite(predicted_image_path, img)

        self.save_traffic_lights(img, result, self.image_path, self.timestamp)

        return img, self.results


In [36]:
model = YOLO('../models/yolo/yolov8m.pt')
image_directory = '../data/frames/images'

for filename in os.listdir(image_directory):
    if filename.endswith("_original.jpg"):
        image_path = os.path.join(image_directory, filename)
        pred = PredPipe(model, image_path, 0, 0)
        
        try:
            # Tente predizer os semáforos na imagem
            pred.predict_image(add_noise=False)
        except ValueError as e:
            # Se nenhum semáforo for encontrado, imprima o erro
            print(f"Erro na imagem {filename}: {str(e)}")


image 1/1 c:\Users\Thiago\OneDrive\Documentos\Facul\6 Sexto semestre\TCC\TCC-CDIA\notebooks\..\data\frames\images\100_original.jpg: 384x640 6 cars, 2 traffic lights, 541.5ms
Speed: 3.0ms preprocess, 541.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)



Semáforo 1 da imagem 100_original já existe em ../data/processed/outputs/semafaros/standard/traffic_light_100_original.jpg. Não foi salvo novamente.
Semáforo 2 da imagem 100_original já existe em ../data/processed/outputs/semafaros/standard/traffic_light_100_original(1).jpg. Não foi salvo novamente.


image 1/1 c:\Users\Thiago\OneDrive\Documentos\Facul\6 Sexto semestre\TCC\TCC-CDIA\notebooks\..\data\frames\images\101_original.jpg: 384x640 10 persons, 4 cars, 3 traffic lights, 525.6ms
Speed: 2.0ms preprocess, 525.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)



Semáforo 1 da imagem 101_original já existe em ../data/processed/outputs/semafaros/standard/traffic_light_101_original.jpg. Não foi salvo novamente.
Semáforo 2 da imagem 101_original já existe em ../data/processed/outputs/semafaros/standard/traffic_light_101_original(1).jpg. Não foi salvo novamente.
Semáforo 3 da imagem 101_original já existe em ../data/processed/outputs/semafaros/standard/traffic_light_101_original(2).jpg. Não foi salvo novamente.


image 1/1 c:\Users\Thiago\OneDrive\Documentos\Facul\6 Sexto semestre\TCC\TCC-CDIA\notebooks\..\data\frames\images\102_original.jpg: 384x640 6 cars, 3 traffic lights, 517.6ms
Speed: 3.0ms preprocess, 517.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)



...

Semáforo 1 da imagem 98_original já existe em ../data/processed/outputs/semafaros/standard/traffic_light_98_original.jpg. Não foi salvo novamente.
Semáforo 2 da imagem 98_original já existe em ../data/processed/outputs/semafaros/standard/traffic_light_98_original(1).jpg. Não foi salvo novamente.


image 1/1 c:\Users\Thiago\OneDrive\Documentos\Facul\6 Sexto semestre\TCC\TCC-CDIA\notebooks\..\data\frames\images\99_original.jpg: 384x640 4 cars, 3 traffic lights, 490.8ms
Speed: 2.0ms preprocess, 490.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)



Semáforo 1 da imagem 99_original já existe em ../data/processed/outputs/semafaros/standard/traffic_light_99_original.jpg. Não foi salvo novamente.
Semáforo 2 da imagem 99_original já existe em ../data/processed/outputs/semafaros/standard/traffic_light_99_original(1).jpg. Não foi salvo novamente.
Semáforo 3 da imagem 99_original já existe em ../data/processed/outputs/semafaros/standard/traffic_light_99_original(2).jpg. Não foi salvo novamente.


image 1/1 c:\Users\Thiago\OneDrive\Documentos\Facul\6 Sexto semestre\TCC\TCC-CDIA\notebooks\..\data\frames\images\9_original.jpg: 384x640 9 cars, 1 bus, 1 truck, 497.7ms
Speed: 2.0ms preprocess, 497.7ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)


Erro na imagem 9_original.jpg: Nenhum semáforo detectado na imagem.


In [37]:
import os
import cv2
import numpy as np

def add_noise_to_images(input_folder, output_folder, noise_pctg, suffix='noisy'):
    # Certifique-se de que a pasta de saída exista
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Listar arquivos na pasta de entrada
    image_files = os.listdir(input_folder)

    for image_file in image_files:
        if image_file.endswith('.jpg'):
            # Carregar a imagem original
            image_path = os.path.join(input_folder, image_file)
            image = cv2.imread(image_path)
            
            # Aplicar ruído à imagem inteira
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = image.astype(np.float32) / 255.0
            noise = np.random.uniform(low=0.0, high=1.0, size=image.shape)
            noise_mask = np.random.binomial(1, noise_pctg, size=image.shape[:-1])
            noise_mask = np.expand_dims(noise_mask, axis=-1)
            noise_mask = np.repeat(noise_mask, 3, axis=-1)
            image = noise_mask * noise + (1 - noise_mask) * image
            image = (image * 255.0).astype(np.uint8)

            # Salvar a imagem com o novo nome
            noisy_image_file = f"{os.path.splitext(image_file)[0]}_{suffix}.jpg"
            noisy_image_path = os.path.join(output_folder, noisy_image_file)
            cv2.imwrite(noisy_image_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
            print(f"Imagem {image_file} com ruído aplicado e salva como {noisy_image_file}")

# Exemplo de uso
input_folder = "../data/processed/outputs/semafaros/standard"
output_folder = "../data/processed/outputs/semafaros/noisy"
noise_pctg = 0.3  # Porcentagem de ruído a ser aplicada
add_noise_to_images(input_folder, output_folder, noise_pctg)


Imagem traffic_light_100_original(1).jpg com ruído aplicado e salva como traffic_light_100_original(1)_noisy.jpg
Imagem traffic_light_100_original.jpg com ruído aplicado e salva como traffic_light_100_original_noisy.jpg
Imagem traffic_light_101_original(1).jpg com ruído aplicado e salva como traffic_light_101_original(1)_noisy.jpg
Imagem traffic_light_101_original(2).jpg com ruído aplicado e salva como traffic_light_101_original(2)_noisy.jpg
Imagem traffic_light_101_original.jpg com ruído aplicado e salva como traffic_light_101_original_noisy.jpg
Imagem traffic_light_102_original(1).jpg com ruído aplicado e salva como traffic_light_102_original(1)_noisy.jpg
Imagem traffic_light_102_original(2).jpg com ruído aplicado e salva como traffic_light_102_original(2)_noisy.jpg
Imagem traffic_light_102_original.jpg com ruído aplicado e salva como traffic_light_102_original_noisy.jpg
Imagem traffic_light_103_original(1).jpg com ruído aplicado e salva como traffic_light_103_original(1)_noisy.jpg
I